In [1]:
from model import HPE, NME, lr_scheduler
import utils
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import cv2
import math

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# To solve "No algorithm worked!", run this codes.

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
df = pd.read_csv('data/mpii_dataset.csv')
df = df.drop(['Unnamed: 0', 'Activity', 'Category', 'Scale'], axis=1)
df.head(5)

,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,...,r elbow_X,r elbow_Y,r shoulder_X,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y
0,015601864.jpg,620,394,616,269,573,185,647,188,661,...,553,161,601,167,692,185,693,240,688,313
1,015599452.jpg,-1,-1,-1,-1,806,543,720,593,-1,...,555,410,647,281,719,299,711,516,545,466
2,005808361.jpg,804,711,816,510,908,438,1040,454,906,...,798,340,902,253,1067,253,1167,353,1142,478
3,086617615.jpg,301,461,305,375,201,340,294,342,335,...,260,335,244,261,314,264,327,320,362,346
4,060111501.jpg,980,322,896,318,865,248,943,226,948,...,754,247,792,147,923,123,995,163,961,223


In [5]:
data_list = df.to_numpy()
image_list = data_list[:,0].astype(np.str)
label_list = data_list[:,1:].astype(np.int32)
data_list = np.column_stack((image_list, label_list))

In [6]:
def data_generator(data_list):
    np.random.shuffle(data_list)
    for data in data_list:
        image_path = data[0]
        joint_coor = data[1:].astype(np.int32)
        img = tf.io.read_file('data/images/' + image_path.decode('utf-8'))
        img = tf.image.decode_jpeg(img, channels=3)
        img_shape = img.shape
        img = tf.image.resize(img, [256, 256])

        tmp = []
        # normalize and reshape
        for i in range(16):
            if joint_coor[i*2] == -1:
                tmp.append([-1, -1])
            else:
                tmp.append([joint_coor[i*2] / img_shape[1] * 64, joint_coor[i*2+1] / img_shape[0] * 64])

        joint = np.array(tmp, dtype=np.int32)
        hm = utils.create_target_heatmap(np.array([joint]))
        hm = tf.convert_to_tensor(hm, dtype=tf.float32, name='heatmap')

        yield (tf.reshape(img, [1, 256, 256, 3]) / 255, hm)

In [7]:
# sum_of_hm = np.zeros([hm.shape[1], hm.shape[2]])
# print(hm.shape[-1])
# for i in range(hm.shape[-1]):
#     sum_of_hm = np.add(sum_of_hm, hm[:,:,:,i])
# plt.imshow(sum_of_hm[0])

In [8]:
# plt.imshow(img)

In [9]:
model = HPE()

Model: "HPE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(1, 256, 256, 3)]   0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (1, 128, 128, 64)    9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (1, 128, 128, 64)    256         conv2d[0][0]                     
__________________________________________________________________________________________________
tf.nn.relu (TFOpLambda)         (1, 128, 128, 64)    0           batch_normalization[0][0]        
________________________________________________________________________________________________

In [10]:
# checkpoint

checkpoint_path = "data/checkpoint/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

model.save_weights(checkpoint_path.format(epoch=0))

In [11]:
np.random.shuffle(data_list)
split_ratio = int(data_list.shape[0]*0.8)
train_list = data_list[:split_ratio]
val_list = data_list[split_ratio:]

In [12]:
train = tf.data.Dataset.from_generator(data_generator,
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes=([None, 256, 256, 3], [None, 64, 64, 16]),
                                         args=(train_list, ))

val = tf.data.Dataset.from_generator(data_generator,
                                         output_types=(np.float32, np.float32),
                                         output_shapes=([None, 256, 256, 3], [None, 64, 64, 16]),
                                         args=(val_list, ))

In [13]:
lr = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
opt = tf.keras.optimizers.RMSprop(
    learning_rate=1e-04, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop')
model.compile(
    optimizer=opt, loss=NME, metrics=None, loss_weights=None,
    sample_weight_mode=None, weighted_metrics=None, target_tensors=None,
    distribute=None)
model.build((None, 256, 256, 3))

In [14]:
hist = model.fit(train, validation_data=val, batch_size=1, callbacks=[lr, cp_callback], epochs=50)

Epoch 1/50
13897/13897 [==============================] - 1401s 92ms/step - loss: 0.0097 - val_loss: 0.0020

Epoch 00001: saving model to data/checkpoint/cp-0001.ckpt
Epoch 2/50
13897/13897 [==============================] - 1247s 90ms/step - loss: 0.0018 - val_loss: 0.0020

Epoch 00002: saving model to data/checkpoint/cp-0002.ckpt
Epoch 3/50
13897/13897 [==============================] - 1251s 90ms/step - loss: 0.0016 - val_loss: 0.0018

Epoch 00003: saving model to data/checkpoint/cp-0003.ckpt
Epoch 4/50
13897/13897 [==============================] - 1252s 90ms/step - loss: 0.0015 - val_loss: 0.0020

Epoch 00004: saving model to data/checkpoint/cp-0004.ckpt
Epoch 5/50
13897/13897 [==============================] - 1251s 90ms/step - loss: 0.0014 - val_loss: 0.0019

Epoch 00005: saving model to data/checkpoint/cp-0005.ckpt
Epoch 6/50
13897/13897 [==============================] - 1248s 90ms/step - loss: 0.0013 - val_loss: 0.0020

Epoch 00006: saving model to data/checkpoint/cp-0006.ckp

Epoch 49/50
13897/13897 [==============================] - 1232s 89ms/step - loss: 1.0463e-04 - val_loss: 0.0027

Epoch 00049: saving model to data/checkpoint/cp-0049.ckpt
Epoch 50/50
13897/13897 [==============================] - 1232s 89ms/step - loss: 1.0595e-04 - val_loss: 0.0035

Epoch 00050: saving model to data/checkpoint/cp-0050.ckpt
